In [12]:
import numpy as np
import keras
from keras.utils import get_file
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.applications import vgg19
from keras.utils import plot_model
from keras import Model
import tensorflow as tf
from keras.optimizers import SGD
from PIL import Image

In [4]:
style_image_path = "./style.jpg"
content_image_path = "./content.jpg"

In [ ]:
a = plt.imread(content_image_path)
b = plt.imread(style_image_path)
f, axarr = plt.subplots(1,2, figsize=(15,15))
axarr[0].imshow(a)
axarr[1].imshow(b)
plt.show()

In [9]:
# Funciones útiles

def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def content_cost(content, combination):
    return tf.reduce_sum(tf.square(combination - content))

def style_cost(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

def loss_function(combination_image, content_image, style_reference_image, content_layer, style_layers, content_weight, style_weight):

    # 1. Combinar todas las imágenes en un mismo tensor.
    input_tensor = tf.concat(
        [content_image, style_reference_image, combination_image], axis=0
    )

    # 2. Obtener los valores en todas las capas para las tres imágenes
    features = feature_extractor(input_tensor)

    #3. Inicializar el coste
    loss = tf.zeros(shape=())

    # 4. Extraer las capas de contenido + coste de contenido
    layer_features = features[content_layer]
    content_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]

    loss = loss + content_weight * content_cost(content_image_features, combination_features
    )
    # 5. Extraer las capas de estilo + coste de estilo
    for layer_name in style_layers:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_cost(style_reference_features, combination_features)
        loss += (style_weight / len(style_layers)) * sl

    return loss

def compute_loss_and_grads(combination_image, base_image, style_reference_image, content_layer, style_layers, content_weight, style_weight):
    with tf.GradientTape() as tape:
        loss = loss_function(combination_image, base_image, style_reference_image, content_layer, style_layers, content_weight, style_weight)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

def preprocess_image(image_path):
    # Util function to open, resize and format pictures into appropriate tensors
    img = keras.preprocessing.image.load_img(
        image_path, target_size=(img_nrows, img_ncols)
    )
    img = keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)

def deprocess_image(x):

    # Convertimos el tensor en Array
    x = x.reshape((img_nrows, img_ncols, 3))

    # Hacemos que no tengan promedio 0
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68

    # Convertimos de BGR a RGB.
    x = x[:, :, ::-1]

    # Nos aseguramos que están entre 0 y 255
    x = np.clip(x, 0, 255).astype("uint8")

    return x

In [ ]:
# Usamos el modelo preentrenado VGG19

model = vgg19.VGG19(weights="imagenet", include_top=False)

model.summary()

outputs_dict= dict([(layer.name, layer.output) for layer in model.layers])

# Armamos un modelo donde podamos acceder a todas las salidas de todas las capas
feature_extractor = Model(inputs=model.inputs, outputs=outputs_dict)

# Las capas a usar para el cálculo del error de estilo
capas_estilo = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]

# La capa a usar para el cálculo del error de contenido
capas_contenido = "block5_conv2"

content_weight = 2.5e-8
style_weight = 1e-6

In [11]:
# Preparamos la imagen

width, height = keras.preprocessing.image.load_img(content_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

optimizer = SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

content_image = preprocess_image(content_image_path)
style_reference_image = preprocess_image(style_image_path)

# Partimos de la imagen de contenido, pero podría ser una imagen al azar
combination_image = tf.Variable(preprocess_image(content_image_path))

In [ ]:
# "Entrenamos la imagen resultante"

iterations = 4000

for i in range(0, iterations):
    loss, grads = compute_loss_and_grads(
        combination_image, content_image, style_reference_image,
        capas_contenido, capas_estilo, content_weight, style_weight
    )
    print("Iteration %d: loss=%.2f" % (i+1, loss))

    optimizer.apply_gradients([(grads, combination_image)])

    if (i+1) % 10 == 0:
        img = deprocess_image(combination_image.numpy())
        im = Image.fromarray(img[..., ::-1])
        display(im)